# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-22 11:48:04] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-22 11:48:04] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-22 11:48:04] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-22 11:48:05] INFO lfm2.py:102: Registered SGLang Lfm2Config to override HuggingFace's version


[2026-01-22 11:48:07] INFO server_args.py:1724: Attention backend not specified. Use fa3 backend by default.


...

Capturing batches (bs=128 avail_mem=9.32 GB):   0%|          | 0/20 [00:00<?, ?it/s]

Capturing batches (bs=72 avail_mem=1.78 GB):  25%|██▌       | 5/20 [00:06<00:13,  1.10it/s]

Capturing batches (bs=24 avail_mem=1.76 GB):  60%|██████    | 12/20 [00:06<00:02,  3.54it/s]

Capturing batches (bs=1 avail_mem=1.73 GB):  95%|█████████▌| 19/20 [00:06<00:00,  7.48it/s]

Capturing batches (bs=1 avail_mem=1.73 GB): 100%|██████████| 20/20 [00:06<00:00,  3.04it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Cesar, I'm from Canada and I am a retired teacher, I was always good with children. I was always busy teaching and always had a lot of students. I used to teach in Ontario, Canada. I was a good teacher and I was never late, I was very patient, I was always nice to the students, I always knew how to deal with any situation. I used to do lots of socializing and traveling around the world. I visited China, Japan, Australia, and other countries. I was an expert on the teaching techniques in the United States, which was the United States, and I had a lot of
Prompt: The president of the United States is
Generated text:  a symbolic figurehead of what?
A. The executive branch
B. The legislative branch
C. The judicial branch
D. The military

The president of the United States is a symbolic figurehead, serving as both the head of state and the head of government. As the head of state, they represent the country in international relations and are respons

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I'm a [Age] year old [Gender] [Occupation]. I'm a [Skill/Ability] who has been [Number of Years] years in the field of [Field of Interest]. I'm [Favorite Color] and [Favorite Food] and I love [Favorite Activity]. I'm [Favorite Book] and [Favorite Movie]. I'm [Favorite Music] and [Favorite Sport]. I'm [Favorite Animal] and [Favorite Pet]. I'm [Favorite Place] and [Favorite Person]. I'm [Favorite Thing to Do]. I'm [Favorite Quote]. I'm [Favorite Movie Review].

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, also known as "La Ville Flottante" or "La Ville Blanche". It is the largest city in France and the second-largest city in the European Union. Paris is home to many famous landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. The city is also known for its rich cultural heritage, including the Notre-Dame Cathedral, the Louvre Museum, and the Champs-Élysées. Paris is a vibrant and diverse city with a rich history and a thriving economy. It is a popular tourist destination and a major cultural and economic center in Europe. The city

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by a number of trends that are expected to shape the technology's direction and impact on society. Here are some of the most likely trends that are expected to shape the future of AI:

1. Increased integration with human intelligence: AI is likely to become more integrated with human intelligence, allowing machines to learn from and adapt to human behavior and experiences. This integration could lead to more sophisticated and personalized AI systems that can better understand and respond to human needs and preferences.

2. Greater emphasis on ethical and social considerations: As AI becomes more integrated with human intelligence, there will be a greater emphasis on ethical and social considerations. This



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  ____. I'm a/an ____. I love to ____. I have a/an ____. What are some of your hobbies? (list of hobbies can be filled in) (list of hobbies can be filled in) Where do you work? (list of places can be filled in)
I'm a/an ____. I love to ____. I have a/an ____. What are some of your hobbies? (list of hobbies can be filled in) (list of hobbies can be filled in) Where do you work? (list of places can be filled in) 
Hello, my name is [insert name] and I'm a

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris. 
(A) In 2023, Paris is 17.3 million people.
(B) Paris has been the capital of France since the 12th century.
(C) Paris has 2700 years of history.
(D) Paris is France's largest city. (E) In 2023, Paris has 17 million people. To determine which statement is correct, I will analyze each option step-by-step:

(A)

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Your

 Name

],

...

 is

 the

 integration

 of

 natural

In [6]:
llm.shutdown()